In [ ]:
def is_colision(self, x, y):
        for cone in self.car.forward_cones:
            ex = cone.x - x; ey = cone.y - y
            if ex**2 + ey**2 - cone.radius**2 <= 0:
                return True
        return False

    def sense_obstacle(self):
        data = []
        points = []
        x = self.car.get_position()[0] + self.pos[0]
        y = self.car.get_position()[1] + self.pos[1]
        for angle in np.linspace(self.angle_min + self.car.state.yaw, self.angle_max + self.car.state.yaw,self.angle_num, True):
            sense = False
            x1 = x + self.range_min*math.cos(angle)
            y1 = y + self.range_min*math.sin(angle)

            x2 = x + self.range_max*math.cos(angle)
            y2 = y + self.range_max*math.sin(angle)

            for i in range(self.range_num+1):
                u = i/self.range_num
                x3 = x2*u + x1*(1 - u)
                y3 = y2*u + y1*(1 - u)
                if self.is_colision(x3, y3):
                    dis = self.distance((x3, y3))
                    data.append({'position': (x3, y3), 'distance': dis, 'angle': angle})
                    points.append((x3, y3))
                    sense = True
                    break
            if not sense:
                data.append({'position': None, 'distance': self.range_max, 'angle': angle})
        self.sense_data = data
        self.points = points

In [ ]:
def sense_obstacle_fast_angles(self, track):

        data = []
        points = []

        for angle in np.linspace(self.angle_min, self.angle_max, self.angle_num, True):
            direction = np.array([math.cos(angle), math.sin(angle)])
            closest_point = None
            min_dist = self.range_max

            for cone in self.car.forward_cones_lidar:
                circle_center = np.array([cone.x, cone.y])
                r = cone.radius
                hit = self.ray_circle_intersection(self.pos, direction, circle_center, r)
                if hit is not None:
                    dist = np.linalg.norm(hit - self.pos)
                    if dist < min_dist:
                        min_dist = dist
                        closest_point = hit
                        color = cone.color

            if closest_point is not None:
                data.append({'position': tuple(closest_point), 'distance': min_dist, 'angle': angle, 'color': color})
                points.append(tuple(closest_point))
            else:
                data.append({'position': None, 'distance': self.range_max, 'angle': angle, 'color': None})

        self.sense_data = data
        self.points = points


In [ ]:
def find_visible_cones(self):

        visible_cones = []

        forward_vec = np.array([np.cos(self.sense_yaw), np.sin(self.sense_yaw)])

        # Step 2: Check distance to each cone
        for cone in self.car.track.cones:

            dx = cone.x - self.car.state.x
            dy = cone.y - self.car.state.y
            dist = math.hypot(dx, dy)

            direct_to_cone = np.array([dx,dy])/dist
            dot_product = np.clip(np.dot(forward_vec, direct_to_cone), -1.0, 1.0)
            angle_deg = np.degrees(np.arccos(dot_product))


            if dist <= self.range_max and angle_deg <= 85:
                visible_cones.append(cone)
                self.seen_cones.add(cone)

        self.visible_cones = visible_cones

    def distance(self, obs_pose):
        ex = obs_pose[0] - self.pos[0]
        ey = obs_pose[1] - self.pos[1]
        return math.hypot(ex, ey)

In [ ]:
def fast_euclidean_clustering(self, distance_threshold=0.5, min_cluster_size=1):
        points = self.points
        tree = cKDTree(points)
        visited = np.zeros(len(points), dtype=bool)
        clusters = []

        for i in range(len(points)):
            if visited[i]:
                continue

            queue = [i]
            cluster = []

            while queue:
                idx = queue.pop()
                if visited[idx]:
                    continue
                visited[idx] = True
                cluster.append(idx)
                neighbors = tree.query_ball_point(points[idx], distance_threshold)
                queue.extend([n for n in neighbors if not visited[n]])

            if len(cluster) >= min_cluster_size:
                clusters.append(cluster)

        return clusters

In [ ]:
def plot_detections(self, clusters, subplot_index=111):
        # Map positions to indices in self.sense_data
        pos_to_data_index = {
            tuple(d['position']): i
            for i, d in enumerate(self.sense_data)
            if d['position'] is not None
        }

        # Generate a color for each cluster
        colors = cm.rainbow(np.linspace(0, 1, len(clusters)))

        ax = plt.subplot(subplot_index, polar=True)

        for cluster, color in zip(clusters, colors):
            cluster_angles = []
            cluster_distances = []
            for idx in cluster:
                pos = self.points[idx]
                data_idx = pos_to_data_index.get(tuple(pos))
                if data_idx is not None:
                    d = self.sense_data[data_idx]
                    cluster_angles.append(d['angle'])
                    cluster_distances.append(d['distance'])

            ax.scatter(cluster_angles, cluster_distances, s=10, color=color,
                    label=f'Cluster {clusters.index(cluster)}')

        ax.set_title("Clustered LiDAR Scan (Polar)")
        ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

In [ ]:
    def estimate_cone_center(self, cluster_indices, known_radius=0.1):
        cluster = np.array([self.points[i] for i in cluster_indices])
        n = len(cluster)

        if n == 0:
            return None

        elif n == 1:
            idx = cluster_indices[0]
            point = self.points[idx]
            sense_idx = self.find_matching_sense_idx(point)
            sense = self.sense_data[sense_idx]
            x, y = sense['position']
            angle = sense['angle']
            dx = math.cos(angle)
            dy = math.sin(angle)
            return (x - known_radius * dx, y - known_radius * dy)

        elif n == 2:
            p1, p2 = cluster
            midpoint = (p1 + p2) / 2
            vec = p2 - p1
            norm_vec = np.array([-vec[1], vec[0]])  # perpendicular
            norm_vec = norm_vec / np.linalg.norm(norm_vec)
            center1 = midpoint + norm_vec * known_radius
            center2 = midpoint - norm_vec * known_radius
            return tuple(center1 if np.linalg.norm(center1) < np.linalg.norm(center2) else center2)

        else:
            first = complex(*cluster[0])
            middle = complex(*cluster[len(cluster) // 2])
            last = complex(*cluster[-1])
            pos = self.circle_from_3_points(first, middle, last) # pos, r
            return (pos.real, pos.imag)

In [ ]:
def get_detected_cones(self):
        """
        Converts self.cone_pos and cluster color info into a list of Cone objects.
        """
        detected_cones = []

        # Build a mapping from point (x, y) → index in sense_data
        pos_to_index = {
            tuple(d['position']): i
            for i, d in enumerate(self.sense_data)
            if d['position'] is not None
        }

        if self.clusters and self.cone_pos:
            for cluster, pos in zip(self.clusters, self.cone_pos):
                if pos is None:
                    continue

                # Count dominant color
                color_counts = Counter()
                for idx in cluster:
                    point = tuple(self.points[idx])
                    sense_idx = pos_to_index.get(point)
                    if sense_idx is not None:
                        color = self.sense_data[sense_idx].get('color', 'black')
                        color_counts[color] += 1

                dominant_color = color_counts.most_common(1)[0][0] if color_counts else 'black'
                cone = Cone(pos[0], pos[1], dominant_color)
                detected_cones.append(cone)

        return detected_cones

In [ ]:
def plot_lidar(self):
        plt.figure(figsize=(12, 6))  # wider to fit two plots

        # --- Subplot 1: Cartesian LiDAR View ---
        ax1 = plt.subplot(1, 2, 1)
        for i in range(len(self.sense_data)):
            angle = self.sense_angle_min + self.resolution * i
            x = self.pos[0]
            y = self.pos[1]
            rayx = [x, x + math.cos(angle) * self.sense_data[i]['distance']]
            rayy = [y, y + math.sin(angle) * self.sense_data[i]['distance']]
            ax1.plot(rayx, rayy, '-b', linewidth=0.5)


        # Build a mapping from point (x, y) → index in sense_data
        pos_to_index = {
            tuple(d['position']): i
            for i, d in enumerate(self.sense_data)
            if d['position'] is not None
        }

        # Plot cone centers with dominant cluster color
        if self.clusters and self.cone_pos:
            for cluster, pos in zip(self.clusters, self.cone_pos):
                if pos is None:
                    continue

                # Use original sense_data indices to get colors
                color_counts = Counter()
                for idx in cluster:
                    point = tuple(self.points[idx])
                    sense_idx = pos_to_index.get(point)
                    if sense_idx is not None:
                        color = self.sense_data[sense_idx].get('color', 'black')
                        color_counts[color] += 1

                dominant_color = color_counts.most_common(1)[0][0] if color_counts else 'black'
                circle = plt.Circle(pos, 0.1, color=dominant_color, fill=True)
                ax1.add_patch(circle)

        if self.car:
            ax1.scatter(self.car.state.x, self.car.state.y, c='purple', marker='s', label="Car")

        if self.car:
            ax1.scatter(self.pos[0], self.pos[1], c='cyan', marker='s', label="Lidar")

        ax1.set_title("LiDAR Vision (Cartesian)")
        ax1.set_aspect('equal')
        ax1.legend()

        # --- Subplot 2: Polar Cluster View ---
        self.plot_detections(self.clusters, subplot_index=122)

        plt.tight_layout()
        plt.show()

In [ ]:
def sense_obstacle_fast_cones(self, track):

        data = []
        visible_cones = []

        angles = np.linspace(self.sense_angle_min, self.sense_angle_max, self.angle_num, True)
        

        for angle in angles:
            data.append({
                'position': None,
                'distance': self.range_max,
                'angle': angle,
                'color': None
            })

        for cone in track.cones:

            cone = 
            
            vec_to_cone = np.array([cone.x - self.pos[0], cone.y - self.pos[1]])
            dist = np.linalg.norm(vec_to_cone)

            if dist > self.range_max:
                continue
            
            angle_to_cone = np.arctan2(vec_to_cone[1], vec_to_cone[0])
            relative_angle = angle_to_cone - self.sense_yaw
            relative_angle = np.arctan2(np.sin(relative_angle), np.cos(relative_angle))

            if (self.angle_max < relative_angle) or (relative_angle < self.angle_min):
                continue
            
            #visible_cones.append(cone)
            #self.seen_cones.add(cone)

            angular_radius = np.arcsin(cone.radius / dist)
            lower_bound = relative_angle - angular_radius
            upper_bound = relative_angle + angular_radius

            candidate_indices = np.where((angles-self.sense_yaw >= lower_bound) & (angles-self.sense_yaw <= upper_bound))[0]

            for idx in candidate_indices:
                ray_angle = angles[idx]
                ray_dir = np.array([np.cos(ray_angle), np.sin(ray_angle)])

                hit = self.ray_circle_intersection(self.pos, ray_dir, np.array([cone.x, cone.y]), cone.radius)
                
                if hit is not None:
                    hit_dist = np.linalg.norm(hit - self.pos)
                    if hit_dist < data[idx]['distance']:
                        # Update the stored data immediately
                        data[idx]['distance'] = hit_dist
                        data[idx]['position'] = tuple(hit)
                        data[idx]['color'] = cone.color

        self.sense_data = data

        # Convert sensed points to car frame
        self.relative_sense_data = []
        for d in data:
            if d['position'] is None:
                self.relative_sense_data.append({
                    'position': None,
                    'distance': self.range_max,
                    'angle': d['angle']-self.sense_yaw,
                    'color': None
                })
                continue
            px, py = d['position']
            dx = px - self.pos[0]
            dy = py - self.pos[1]

            # Rotate by -yaw (from world to car frame)
            cos_yaw = np.cos(-self.sense_yaw)
            sin_yaw = np.sin(-self.sense_yaw)
            rel_x = cos_yaw * dx - sin_yaw * dy
            rel_y = sin_yaw * dx + cos_yaw * dy

            self.relative_sense_data.append({
                'position': (rel_x, rel_y),
                'distance': d['distance'],
                'angle': d['angle']-self.sense_yaw,
                'color': d['color']
            })

        self.points = {
        'positions': [d['position'] for i, d in enumerate(self.relative_sense_data) if d['position'] is not None],
        'indices': [i for i, d in enumerate(self.relative_sense_data) if d['position'] is not None]
        }

In [ ]:
# def match_cones(self, max_dist=0.3):
    #     raw_matches = []
    #     distances = []

    #     for i, c_curr in enumerate(self.lidar.relative_cones):
    #         best_match = None
    #         min_dist = float('inf')
    #         for j, c_prev in enumerate(self.prev_lidar_data):
    #             if c_curr.color != c_prev.color:
    #                 continue
    #             dist = np.hypot(c_curr.x - c_prev.x, c_curr.y - c_prev.y)
    #             if dist < min_dist and dist < max_dist:
    #                 best_match = j
    #                 min_dist = dist
    #         if best_match is not None:
    #             raw_matches.append((best_match, i))  # (prev_idx, curr_idx)
    #             distances.append(min_dist)

    #     # Filter using 2 standard deviations above the mean
    #     if distances:
    #         mean = np.mean(distances)
    #         std_dev = np.std(distances)
    #         threshold = mean + 2 * std_dev
    #         filtered_matches = [
    #             match for match, dist in zip(raw_matches, distances) if dist <= threshold
    #         ]
    #     else:
    #         filtered_matches = []

    #     return filtered_matches


    # def match_lidar_points(self, max_dist=0.3):
    #     raw_matches = []
    #     distances = []

    #     for i, curr in enumerate(self.lidar.relative_sense_data):  # current frame
    #         best_match = None
    #         min_dist = float('inf')
    #         curr_x, curr_y = curr['position']

    #         for j, prev in enumerate(self.prev_lidar_data):  # previous frame
    #             prev_x, prev_y = prev['position']
    #             dist = np.hypot(curr_x - prev_x, curr_y - prev_y)
    #             if dist < min_dist and dist < max_dist:
    #                 best_match = j
    #                 min_dist = dist

    #         if best_match is not None:
    #             raw_matches.append((best_match, i))  # (prev_idx, curr_idx)
    #             distances.append(min_dist)

    #     # Filter using mean + 2 std dev
    #     if distances:
    #         mean = np.mean(distances)
    #         std_dev = np.std(distances)
    #         threshold = mean + 2 * std_dev
    #         filtered_matches = [
    #             match for match, dist in zip(raw_matches, distances) if dist <= threshold
    #         ]
    #     else:
    #         filtered_matches = []

    #     return filtered_matches

    

    # def estimate_transform(self, matches):
    #     if len(matches) < 2:
    #         return None

    #     prev_pts = np.array([[self.prev_lidar_data[i].x, self.prev_lidar_data[i].y] for i, _ in matches])
    #     curr_pts = np.array([[self.lidar.relative_cones[j].x, self.lidar.relative_cones[j].y] for _, j in matches])

    #     centroid_prev = np.mean(prev_pts, axis=0)
    #     centroid_curr = np.mean(curr_pts, axis=0)

    #     P = prev_pts - centroid_prev
    #     C = curr_pts - centroid_curr

    #     H = C.T @ P
    #     U, _, Vt = np.linalg.svd(H)
    #     R = Vt.T @ U.T

    #     if np.linalg.det(R) < 0:
    #         Vt[1, :] *= -1
    #         R = Vt.T @ U.T

    #     t = centroid_prev - R @ centroid_curr
    #     yaw = np.arctan2(R[1, 0], R[0, 0])

    #     return t[0], t[1], yaw

    # def estimate_transform(self, matches):
    #     if len(matches) < 2:
    #         return None

    #     # Extract matched points
    #     prev_pts = np.array([
    #         self.prev_lidar_data[i]['position'] for i, _ in matches
    #     ])
    #     curr_pts = np.array([
    #         self.lidar.relative_sense_data[j]['position'] for _, j in matches
    #     ])

    #     centroid_prev = np.mean(prev_pts, axis=0)
    #     centroid_curr = np.mean(curr_pts, axis=0)

    #     P = prev_pts - centroid_prev
    #     C = curr_pts - centroid_curr

    #     H = C.T @ P
    #     U, _, Vt = np.linalg.svd(H)
    #     R = Vt.T @ U.T

    #     if np.linalg.det(R) < 0:
    #         Vt[1, :] *= -1
    #         R = Vt.T @ U.T

    #     t = centroid_prev - R @ centroid_curr
    #     yaw = np.arctan2(R[1, 0], R[0, 0])

    #     return t[0], t[1], yaw



    # def update_pose(self, dx, dy, dyaw):
    #     R_global = np.array([
    #         [np.cos(self.state_e.yaw), -np.sin(self.state_e.yaw)],
    #         [np.sin(self.state_e.yaw),  np.cos(self.state_e.yaw)]
    #     ])
    #     delta_global = R_global @ np.array([dx, dy])
    #     x_new = self.state_e.x + delta_global[0]
    #     y_new = self.state_e.y + delta_global[1]
    #     yaw_new = self.state_e.yaw + dyaw

    #     self.state_e.x = x_new
    #     self.state_e.y = y_new
    #     self.state_e.yaw = yaw_new


    # def extract_transform(self, transformation):
    #     """
    #     Extract the translation (dx, dy) and yaw (dyaw) from the 3x3 transformation matrix.
    #     """
    #     # Extract rotation and translation
    #     R = transformation[:2, :2]
    #     t = transformation[:2, 2]

    #     # Compute yaw from the rotation matrix
    #     yaw = np.arctan2(R[1, 0], R[0, 0])

    #     # dx, dy are the translation components
    #     dx, dy = t[0], t[1]

    #     return dx, dy, yaw



    
    # def estimate_transform_icp(self, prev_pts, curr_pts, max_iterations=50, threshold=1e-6):
    #     """
    #     Perform 2D ICP (Iterative Closest Point) to estimate the transform between two point clouds.
    #     """
    #     # Start with identity transformation
    #     transformation = np.eye(3)
        
    #     for iteration in range(max_iterations):
    #         # Find closest points in current to previous
    #         closest_points = self.find_closest_points(prev_pts, curr_pts)
            
    #         # Estimate the rigid transformation (rotation and translation)
    #         R, t = self.compute_rigid_transform(prev_pts, closest_points)
            
    #         # Apply the transformation to the current points
    #         curr_pts = self.apply_transformation_to_points(curr_pts, R, t)
            
    #         # Compute the new transformation matrix
    #         transformation_new = np.eye(3)
    #         transformation_new[:2, :2] = R
    #         transformation_new[:2, 2] = t
            
    #         # Check for convergence
    #         if np.linalg.norm(transformation_new - transformation) < threshold:
    #             break
                
    #         transformation = transformation_new

    #     return transformation  # 3x3 transformation matrix (rotation + translation)

    # def find_closest_points(self, prev_pts, curr_pts):
    #     """
    #     Find the closest points in prev_pts for each point in curr_pts.
    #     """
    #     closest_points = []
    #     for curr_point in curr_pts:
    #         distances = np.linalg.norm(prev_pts - curr_point, axis=1)
    #         closest_idx = np.argmin(distances)
    #         closest_points.append(prev_pts[closest_idx])
    #     return np.array(closest_points)
    
    # def compute_rigid_transform(self, A, B):
    #     """
    #     Compute the rigid transform (rotation and translation) that aligns A to B in 2D.
    #     """
    #     centroid_A = np.mean(A, axis=0)
    #     centroid_B = np.mean(B, axis=0)
        
    #     A_centered = A - centroid_A
    #     B_centered = B - centroid_B
        
    #     H = A_centered.T @ B_centered
    #     U, _, Vt = np.linalg.svd(H)
    #     R = Vt.T @ U.T
        
    #     if np.linalg.det(R) < 0:
    #         Vt[1, :] *= -1
    #         R = Vt.T @ U.T
        
    #     t = centroid_B - R @ centroid_A
        
    #     return R, t


    # def apply_transformation_to_points(self, points, R, t):
    #     """
    #     Apply the transformation (rotation and translation) to a set of points.
    #     """
    #     return np.dot(points, R.T) + t


    # def apply_transformation(self, point, transform):
    #     """
    #     Apply the transformation to a single point.
    #     """
    #     point_homogeneous = np.append(point, 1)  # Convert to homogeneous coordinates
    #     transformed_point = transform @ point_homogeneous
    #     return transformed_point[:2]  # Return only x, y




    # def match_lidar_points(self, max_dist=0.3, max_iterations=50, threshold=1e-6):
    #     """
    #     Match lidar points using ICP method and return matched points.
    #     """
    #     raw_matches = []
    #     distances = []

    #     # Get the previous and current lidar data
    #     prev_points = np.array([cone['position'] for cone in self.prev_lidar_data])  # previous frame
    #     curr_points = np.array([cone['position'] for cone in self.lidar.relative_sense_data])  # current frame
        
    #     # Perform ICP to estimate the transformation
    #     transform = self.estimate_transform_icp(prev_points, curr_points, max_iterations, threshold)

    #     if transform is not None:
    #         # The transformation (dx, dy, dyaw) is embedded in this transformation
    #         dx, dy, dyaw = self.extract_transform(transform)

    #         # Now apply the transformation to the pose
    #         self.update_pose(dx, dy, dyaw)

    #         return raw_matches  # Matching is now done, and the pose is updated.

    #     return []  # Return empty if transformation is not valid

In [ ]:
# def point_based_matching(self, point_pairs):
    #     """
    #     This function is based on the paper "Robot Pose Estimation in Unknown Environments by Matching 2D Range Scans"
    #     by F. Lu and E. Milios.

    #     :param point_pairs: the matched point pairs [((x1, y1), (x1', y1')), ..., ((xi, yi), (xi', yi')), ...]
    #     :return: the rotation angle and the 2D translation (x, y) to be applied for matching the given pairs of points
    #     """

    #     x_mean = 0
    #     y_mean = 0
    #     xp_mean = 0
    #     yp_mean = 0
    #     n = len(point_pairs)

    #     if n == 0:
    #         return None, None, None

    #     for pair in point_pairs:

    #         (x, y), (xp, yp) = pair

    #         x_mean += x
    #         y_mean += y
    #         xp_mean += xp
    #         yp_mean += yp

    #     x_mean /= n
    #     y_mean /= n
    #     xp_mean /= n
    #     yp_mean /= n

    #     s_x_xp = 0
    #     s_y_yp = 0
    #     s_x_yp = 0
    #     s_y_xp = 0
    #     for pair in point_pairs:

    #         (x, y), (xp, yp) = pair

    #         s_x_xp += (x - x_mean)*(xp - xp_mean)
    #         s_y_yp += (y - y_mean)*(yp - yp_mean)
    #         s_x_yp += (x - x_mean)*(yp - yp_mean)
    #         s_y_xp += (y - y_mean)*(xp - xp_mean)

    #     rot_angle = math.atan2(s_x_yp - s_y_xp, s_x_xp + s_y_yp)
    #     translation_x = xp_mean - (x_mean*math.cos(rot_angle) - y_mean*math.sin(rot_angle))
    #     translation_y = yp_mean - (x_mean*math.sin(rot_angle) + y_mean*math.cos(rot_angle))

    #     return rot_angle, translation_x, translation_y

In [ ]:
#  ICP parameters
EPS = 0.0001
MAX_ITER = 100

show_animation = False


def icp_matching(previous_points, current_points):
    """
    Iterative Closest Point matching
    - input
    previous_points: 2D or 3D points in the previous frame
    current_points: 2D or 3D points in the current frame
    - output
    R: Rotation matrix
    T: Translation vector
    """
    H = None  # homogeneous transformation matrix

    dError = np.inf
    preError = np.inf
    count = 0

    if show_animation:
        fig = plt.figure()
        if previous_points.shape[0] == 3:
           fig.add_subplot(111, projection='3d')

    while dError >= EPS:
        count += 1

        if show_animation:  # pragma: no cover
            plot_points(previous_points, current_points, fig)
            plt.pause(0.1)

        indexes, error = nearest_neighbor_association(previous_points, current_points)
        Rt, Tt = svd_motion_estimation(previous_points[:, indexes], current_points)
        # update current points
        current_points = (Rt @ current_points) + Tt[:, np.newaxis]

        dError = preError - error
        print("Residual:", error)

        if dError < 0:  # prevent matrix H changing, exit loop
            print("Not Converge...", preError, dError, count)
            break

        preError = error
        H = update_homogeneous_matrix(H, Rt, Tt)

        if dError <= EPS:
            print("Converge", error, dError, count)
            break
        elif MAX_ITER <= count:
            print("Not Converge...", error, dError, count)
            break

    R = np.array(H[0:-1, 0:-1])
    T = np.array(H[0:-1, -1])

    return R, T

def icp_matching_f(previous_points, current_points):
    """
    Iterative Closest Point matching
    - input
    previous_points: 2D or 3D points in the previous frame
    current_points: 2D or 3D points in the current frame
    - output
    R: Rotation matrix
    T: Translation vector
    """
    H = None  # homogeneous transformation matrix

    dError = np.inf
    preError = np.inf
    count = 0

        # Check if the number of points differs
    n_points_prev = previous_points.shape[1]
    n_points_curr = current_points.shape[1]

    # Match points based on the current set's size
    if n_points_prev > n_points_curr:
        previous_points = previous_points[:, :n_points_curr]  # Down-sample the larger point cloud
    elif n_points_curr > n_points_prev:
        current_points = current_points[:, :n_points_prev]  # Down-sample the larger point cloud


    while dError >= EPS:
        count += 1

        indexes, error = nearest_neighbor_association(previous_points, current_points)
        # Slice based on indexes to align points
        previous_matched_points = previous_points[:, indexes]
        current_matched_points = current_points

        Rt, Tt = svd_motion_estimation_f(previous_matched_points, current_matched_points)

        # Apply the transformation to the current points
        current_points = (Rt @ current_points) + Tt[:, np.newaxis]

        dError = preError - error
        if dError < 0:
            print("Not Converged...", preError, dError, count)
            break

        preError = error
        H = update_homogeneous_matrix(H, Rt, Tt)

        if dError <= EPS:
            print("Converged", error, dError, count)
            break
        elif MAX_ITER <= count:
            print("Not Converged...", error, dError, count)
            break

    R = np.array(H[0:-1, 0:-1])
    T = np.array(H[0:-1, -1])

    return R, T



def svd_motion_estimation_f(previous_points, current_points):
    """
    Estimate the rigid body transformation (rotation and translation) between previous_points and current_points.
    Returns:
    - R: rotation matrix
    - t: translation vector
    """

    
    pm = np.mean(previous_points, axis=1)
    cm = np.mean(current_points, axis=1)

    p_shift = previous_points - pm[:, np.newaxis]
    c_shift = current_points - cm[:, np.newaxis]

    # Compute the covariance matrix
    W = c_shift @ p_shift.T
    u, s, vh = np.linalg.svd(W)

    R = (u @ vh).T
    t = pm - (R @ cm)

    return R, t


def downsample_to_match_size(reference_points, target_points):
    """
    Downsample the larger point cloud so that both have the same number of points.
    
    Inputs:
    - reference_points: np.ndarray of shape (D, N1)
    - target_points: np.ndarray of shape (D, N2)
    
    Returns:
    - reference_points_resized: shape (D, N)
    - target_points_resized: shape (D, N)
    """
    N1 = reference_points.shape[1]
    N2 = target_points.shape[1]
    N = min(N1, N2)

    if N1 > N:
        indices = np.random.choice(N1, N, replace=False)
        reference_points = reference_points[:, indices]
    if N2 > N:
        indices = np.random.choice(N2, N, replace=False)
        target_points = target_points[:, indices]

    return reference_points, target_points




def update_homogeneous_matrix(Hin, R, T):

    r_size = R.shape[0]
    H = np.zeros((r_size + 1, r_size + 1))

    H[0:r_size, 0:r_size] = R
    H[0:r_size, r_size] = T
    H[r_size, r_size] = 1.0

    if Hin is None:
        return H
    else:
        return Hin @ H

def nearest_neighbor_association_f(previous_points, current_points):
    """
    Associate each point in current_points to its nearest neighbor in previous_points.
    Handles differing numbers of points.
    Returns:
    - indexes: indices of closest previous_points
    - error: sum of distances (residual error)
    """

    # KD-Tree for fast NN lookup
    tree = cKDTree(previous_points.T)
    distances, indexes = tree.query(current_points.T)  # Query the nearest neighbors

    # Total ICP residual error
    error = np.sum(distances)

    return indexes, error


def nearest_neighbor_association(previous_points, current_points):

    # calc the sum of residual errors
    delta_points = previous_points - current_points
    d = np.linalg.norm(delta_points, axis=0)
    error = sum(d)

    # calc index with nearest neighbor assosiation
    d = np.linalg.norm(np.repeat(current_points, previous_points.shape[1], axis=1)
                       - np.tile(previous_points, (1, current_points.shape[1])), axis=0)
    indexes = np.argmin(d.reshape(current_points.shape[1], previous_points.shape[1]), axis=1)

    return indexes, error


def svd_motion_estimation(previous_points, current_points):
    pm = np.mean(previous_points, axis=1)
    cm = np.mean(current_points, axis=1)

    p_shift = previous_points - pm[:, np.newaxis]
    c_shift = current_points - cm[:, np.newaxis]

    W = c_shift @ p_shift.T
    u, s, vh = np.linalg.svd(W)

    R = (u @ vh).T
    t = pm - (R @ cm)

    return R, t


def plot_points(previous_points, current_points, figure):
    # for stopping simulation with the esc key.
    plt.gcf().canvas.mpl_connect(
        'key_release_event',
        lambda event: [exit(0) if event.key == 'escape' else None])
    if previous_points.shape[0] == 3:
        plt.clf()
        axes = figure.add_subplot(111, projection='3d')
        axes.scatter(previous_points[0, :], previous_points[1, :],
                    previous_points[2, :], c="r", marker=".")
        axes.scatter(current_points[0, :], current_points[1, :],
                    current_points[2, :], c="b", marker=".")
        axes.scatter(0.0, 0.0, 0.0, c="r", marker="x")
        figure.canvas.draw()
    else:
        plt.cla()
        plt.plot(previous_points[0, :], previous_points[1, :], ".r")
        plt.plot(current_points[0, :], current_points[1, :], ".b")
        plt.plot(0.0, 0.0, "xr")
        plt.axis("equal")

In [ ]:
                #stacked_points = np.vstack((np.array([p['position'] for p in self.prev_lidar_data])[:,0], np.array([p['position'] for p in self.prev_lidar_data])[:,1])) 
                #stacked_points_c = np.vstack((np.array([p['position'] for p in self.lidar.relative_sense_data])[:,0], np.array([p['position'] for p in self.lidar.relative_sense_data])[:,1])) 
                #R, T = icp_matching_f(stacked_points,stacked_points_c)
                #self.apply_transformation_to_pose_d(R,T)
                #self.plot_matched_cones_by_index(matches)

In [ ]:
# Particle and FastSLAM scaffolding based on your LiDAR and car model
import numpy as np
from scipy.stats import multivariate_normal

class Particle:
    def __init__(self, pose, weight=1.0):
        self.pose = np.array(pose)  # [x, y, theta]
        self.weight = weight
        self.landmarks = {}  # {landmark_id: (mean: np.array([x, y]), covariance: np.array([[2x2]]))}

    def predict(self, control_input, dt, motion_noise):
        """
        Apply noisy motion model to update pose
        control_input: [delta_x, delta_y, delta_theta]
        """
        noise = np.random.normal(0, motion_noise, 3)
        dx, dy, dtheta = control_input + noise
        x, y, theta = self.pose
        self.pose = np.array([x + dx, y + dy, theta + dtheta])

    def update_landmark(self, landmark_id, z_rel, R):
        """
        EKF update of landmark (in world frame) given observation z_rel in car frame
        """
        # Transform relative measurement to global coords
        lx = self.pose[0] + np.cos(self.pose[2]) * z_rel[0] - np.sin(self.pose[2]) * z_rel[1]
        ly = self.pose[1] + np.sin(self.pose[2]) * z_rel[0] + np.cos(self.pose[2]) * z_rel[1]
        z = np.array([lx, ly])

        if landmark_id not in self.landmarks:
            self.landmarks[landmark_id] = (z, R)
            return

        mu, sigma = self.landmarks[landmark_id]

        # Measurement prediction is just mu
        H = np.eye(2)
        S = H @ sigma @ H.T + R
        K = sigma @ H.T @ np.linalg.inv(S)
        mu_new = mu + K @ (z - mu)
        sigma_new = (np.eye(2) - K @ H) @ sigma
        self.landmarks[landmark_id] = (mu_new, sigma_new)

        # Likelihood (used for weight update)
        likelihood = multivariate_normal.pdf(z, mean=mu, cov=S)
        self.weight *= likelihood


def initialize_particles(n, init_pose, noise_std):
    return [Particle(pose=init_pose + np.random.normal(0, noise_std, 3)) for _ in range(n)]

def resample_particles(particles):
    weights = np.array([p.weight for p in particles])
    weights /= np.sum(weights)
    indices = np.random.choice(len(particles), size=len(particles), p=weights)
    resampled = [particles[i] for i in indices]
    for p in resampled:
        p.weight = 1.0
    return resampled

In [ ]:
# def compute_mahalanobis_distance(particle, z, lm_id, Q_cov):
#     xf = particle.lm[lm_id].reshape(2, 1)
#     Pf = particle.lmP[2 * lm_id:2 * lm_id + 2]
#     zp, Hv, Hf, Sf = compute_jacobians(particle, xf, Pf, Q_cov)
#     dz = z[0:2].reshape(2, 1) - zp
#     dz[1, 0] = pi_2_pi(dz[1, 0])
#     d2 = float(dz.T @ np.linalg.inv(Sf) @ dz)
#     return d2

# def update_with_observation(particles, z, mahal_thresh=9.21):
#     for iz in range(z.shape[1]):
#         for p in particles:
#             best_lm_id = None
#             best_d2 = float('inf')

#             for lm_id in range(p.lm.shape[0]):
#                 if np.allclose(p.lm[lm_id], 0):
#                     continue

#                 # ✴️ New: Euclidean range filter
#                 dx = p.lm[lm_id, 0] - p.state.x
#                 dy = p.lm[lm_id, 1] - p.state.y
#                 dist = math.hypot(dx, dy)
#                 if dist > MAX_RANGE + 1.0:  # optional buffer (1.0 m)
#                     continue
                
#                 d2 = compute_mahalanobis_distance(p, z[:, iz], lm_id, Q)
#                 if d2 < mahal_thresh and d2 < best_d2:
#                     best_d2 = d2
#                     best_lm_id = lm_id

#             if best_lm_id is not None:
#                 z_with_id = np.array([z[0, iz], z[1, iz], best_lm_id])
#                 w = compute_weight(p, z_with_id, Q)
#                 p.w *= w
#                 p = update_landmark(p, z_with_id, Q)
#                 p = proposal_sampling(p, z_with_id, Q)
#             else:
#                 new_id = np.where(np.all(p.lm == 0, axis=1))[0]
#                 if len(new_id) == 0:
#                     continue
#                 z_with_id = np.array([z[0, iz], z[1, iz], new_id[0]])
#                 p = add_new_lm(p, z_with_id, Q)
#     return particles

In [ ]:
# def update_with_observation(particles, z):
#     for iz in range(len(z[0, :])):
#         landmark_id = int(z[2, iz])

#         for ip in range(N_PARTICLE):
#             # new landmark
#             if abs(particles[ip].lm[landmark_id, 0]) <= 0.01:
#                 particles[ip] = add_new_lm(particles[ip], z[:, iz], Q)
#             # known landmark
#             else:
#                 w = compute_weight(particles[ip], z[:, iz], Q)
#                 particles[ip].w *= w

#                 particles[ip] = update_landmark(particles[ip], z[:, iz], Q)
#                 particles[ip] = proposal_sampling(particles[ip], z[:, iz], Q)

#     return particles

In [ ]:
# def proposal_sampling(particle, z, Q_cov):
#     lm_id = int(z[2])
#     xf = particle.lm[lm_id, :].reshape(2, 1)
#     Pf = particle.lmP[2 * lm_id:2 * lm_id + 2]
#     # State
#     x = np.array([particle.state.x, particle.state.y, particle.state.yaw]).reshape(3, 1)
#     P = particle.P
#     zp, Hv, Hf, Sf = compute_jacobians(particle, xf, Pf, Q_cov)

#     Sfi = np.linalg.inv(Sf)
#     dz = z[0:2].reshape(2, 1) - zp
#     dz[1] = pi_2_pi(dz[1])

#     Pi = np.linalg.inv(P + R)

#     particle.P = np.linalg.inv(Hv.T @ Sfi @ Hv + Pi)  # proposal covariance
#     x += particle.P @ Hv.T @ Sfi @ dz  # proposal mean

#     particle.state.x = x[0, 0]
#     particle.state.y = x[1, 0]
#     particle.state.yaw = x[2, 0]

#     return particle

In [ ]:
# def update_landmark(particle, z, Q_cov):
#     lm_id = int(z[2])
#     xf = np.array(particle.lm[lm_id]).reshape(2, 1)
#     Pf = particle.lmP[lm_id]

#     zp, Hv, Hf, Sf = compute_jacobians(particle, xf, Pf, Q_cov)

#     dz = z[0:2].reshape(2, 1) - zp
#     dz[1, 0] = pi_2_pi(dz[1, 0])

#     xf, Pf = update_kf_with_cholesky(xf, Pf, dz, Q, Hf)

#     particle.lm[lm_id, :] = xf.T
#     particle.lmP[2 * lm_id:2 * lm_id + 2, :] = Pf

#     return particle

In [ ]:
# def add_new_lm(particle, z, Q_cov):
#     r = z[0]
#     b = z[1]
#     lm_id = int(z[2])

#     s = math.sin(pi_2_pi(particle.state.yaw + b))
#     c = math.cos(pi_2_pi(particle.state.yaw + b))

#     particle.lm[lm_id, 0] = particle.state.x + r * c
#     particle.lm[lm_id, 1] = particle.state.y + r * s

#     # covariance
#     dx = r * c
#     dy = r * s
#     d2 = dx ** 2 + dy ** 2
#     d = math.sqrt(d2)
#     Gz = np.array([[dx / d, dy / d],
#                    [-dy / d2, dx / d2]])
#     particle.lmP[2 * lm_id:2 * lm_id + 2] = np.linalg.inv(
#         Gz) @ Q_cov @ np.linalg.inv(Gz.T)

#     return particle